In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import xpress as xp
import pandas as pd

initialisation

In [3]:
meal_data_path = 'meal_data.xlsx'
meal_data = pd.read_excel(meal_data_path)
meal_data.head()

,item_name,course_type,subcategories,meal_type,cost,nf_calories,nf_calories_from_fat,nf_total_fat,nf_saturated_fat,nf_trans_fatty_acid,...,nf_vitamin_a_dv,nf_vitamin_c_dv,nf_calcium_dv,nf_iron_dv,nf_potassium,nf_servings_per_container,nf_serving_size_qty,nf_serving_size_unit,nf_serving_weight_grams,images_front_full_url
0,Trick OR Treat Ice Cream,Soup,cream,NaN,2.08,300.00,140.0,16.0,9.0,0.0,...,8.0,2.0,15.0,4.0,NaN,NaN,4.00,oz scoops,NaN,NaN
1,Sweet Fries Snack,Side,fried,NaN,2.27,1150.00,690.0,79.0,7.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,13.40,oz,NaN,NaN
2,Tomatoes - Salad,Side,red orange,NaN,3.45,9.58,2.4,0.0,0.0,0.0,...,5.0,14.0,0.0,0.0,NaN,NaN,45.52,Grams,45.52,NaN
3,Chicken BBQ Caesar Salad EntrÌ©e,Main dish,poultry,NaN,3.95,203.00,NaN,NaN,2.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.00,servings,NaN,NaN
4,Corn and Black Bean Salad,Side,legume,NaN,3.59,140.00,10.0,1.0,0.0,0.0,...,2.0,4.0,2.0,10.0,NaN,NaN,1.00,Salad,NaN,NaN


In [4]:
score_preference_path = 'customer_dish_preferences.xlsx'
score_preference = pd.read_excel(score_preference_path)
score_preference.set_index('Unnamed: 0', inplace=True)
score_preference.head()

,Dish_1,Dish_2,Dish_3,Dish_4,Dish_5,Dish_6,Dish_7,Dish_8,Dish_9,Dish_10,...,Dish_201,Dish_202,Dish_203,Dish_204,Dish_205,Dish_206,Dish_207,Dish_208,Dish_209,Dish_210
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Customer_1,3,1,1,3,1,2,3,3,1,3,...,3,2,1,3,3,1,3,3,3,1
Customer_2,1,3,3,1,1,3,3,1,3,3,...,2,3,2,3,3,3,3,3,2,3
Customer_3,2,2,1,3,3,3,1,3,3,1,...,3,1,3,3,3,3,3,3,3,3
Customer_4,2,2,1,2,3,2,2,1,3,1,...,3,3,3,2,2,2,2,1,3,3
Customer_5,3,3,3,3,2,2,2,3,3,3,...,2,3,2,3,3,3,2,3,3,2


In [5]:
nutrition_criteria_path = 'nutrition_criteria.xlsx'
nutrition_criteria = pd.read_excel(nutrition_criteria_path)
nutrition_criteria.head()


,ID,Element,F,M,Average
0,1,Calories,1600,2000,1800
1,2,Sodium,2300,2300,2300
2,3,Carbohydrate,130,130,130
3,4,Dietary_fiber,22,28,25
4,5,Protein,46,56,51


In [6]:
table3_path = 'table3.xlsx'
meal_constraints = pd.read_excel(table3_path, sheet_name='meal_constraints', index_col=0)
diet_types = pd.read_excel(table3_path, sheet_name='diet_types', index_col=0)
maximum_servings = pd.read_excel(table3_path, sheet_name='maximum_servings')
day_item = pd.read_excel(table3_path, sheet_name='day_item')
wp = pd.read_excel(table3_path, sheet_name='wp')
wc = pd.read_excel(table3_path, sheet_name='wc')
day_item.head()


,Breakfast,Soup,Salad,Main dish,Side,Appetizer,Dessert
0,1,1,1,1,NaN,NaN,1


In [7]:
# model = xp.problem()
num_item = 210
num_day = 7
num_meal = 3
num_patron = 3
num_course = 7
#num_nutrition = 5

I = list(range(num_item)) # i∈I
M = ['breakfast', 'lunch', 'dinner'] # j∈M
T = list(range(num_day)) # t∈T
P = list(range(num_patron)) # p∈P
L = ['Breakfast', 'Soup', 'Salad', 'Main dish', 'Side', 'Appetizer', 'Dessert'] # l∈L
K = ['calories','cholesterol','sodium','dietary_fiber']
#K = ['calories','cholesterol','dietary_fiber']


l_i = {meal_data.index[i]: meal_data.iloc[i]["course_type"] for i in range(len(meal_data))}
I_l = meal_data.groupby("course_type").apply(lambda x: list(x.index)).to_dict()
S_l = meal_data.groupby("course_type")["subcategories"].unique().apply(list).to_dict()
I_ls = meal_data.groupby(["course_type", "subcategories"]).apply(lambda x: list(x.index)).to_dict()

w = {
    "calories": {i: meal_data.iloc[i]["nf_calories"] for i in I},
    "cholesterol": {i: meal_data.iloc[i]["nf_cholesterol"] for i in I},
    "sodium": {i: meal_data.iloc[i]["nf_sodium"] for i in I},
    "dietary_fiber": {i: meal_data.iloc[i]["nf_dietary_fiber"] for i in I},
}
# w = {
#     "calories": {i: meal_data.iloc[i]["nf_calories"] for i in I},
#     "cholesterol": {i: meal_data.iloc[i]["nf_cholesterol"] for i in I},
#     "dietary_fiber": {i: meal_data.iloc[i]["nf_dietary_fiber"] for i in I},
# }


ns = meal_constraints.to_dict(orient='index')
np = diet_types.to_dict(orient='index')
np_expanded = {p: {j: {l: np[j][l] for l in L} for j in M} for p in P}
nsOWP = maximum_servings.to_dict(orient='records')[0]
nsd = day_item.to_dict(orient='records')[0]
wp_l = wp.to_dict(orient='records')[0]
wc_l = wc.to_dict(orient='records')[0]
pi_raw = score_preference.to_dict(orient='index')
cost = {i: meal_data.loc[i, 'cost'] for i in I}

x = {
    (i, j, t): xp.var(
        vartype=xp.binary,
        name=f'x_{i}_{meal_data.loc[i, "item_name"]}_{j}_{t}'
    )
    for i in I for j in M for t in T
}
y = {
    (i, p, j, t): xp.var(
        vartype=xp.binary,
        name=f'y_{i}_{meal_data.loc[i, "item_name"]}_{p}_{j}_{t}'
    )
    for i in I for p in P for j in M for t in T
}
Q = {
    (p, k, t): xp.var(
        vartype=xp.continuous,
        lb=0,
        name=f'Q_{p}_{k}_{t}'
    )
    for p in P 
    for k in K 
    for t in T
}

alpha_max = {
    p: {
        "calories": 2000,
        "cholesterol": 300,
        "sodium": 2300,
        "dietary_fiber": 1e30 
    }
    for p in P
}
alpha_min = {
    p: {
        "calories": 2000,
        "cholesterol": 0,
        "sodium": 0,
        "dietary_fiber": 28
    }
    for p in P
}
gamma_max = {
    p: {
        "calories": 2000,    
        "cholesterol": 300,
        "sodium": 2300,
        "dietary_fiber": 1e30
    }
    for p in P
}
gamma_min = {
    p: {
        "calories": 2000,    
        "cholesterol": 0,
        "sodium": 0,
        "dietary_fiber": 28
    }
    for p in P
}

C:\Users\18320\AppData\Local\Temp\ipykernel_28272\2645586744.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  I_l = meal_data.groupby("course_type").apply(lambda x: list(x.index)).to_dict()
C:\Users\18320\AppData\Local\Temp\ipykernel_28272\2645586744.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  I_ls = meal_data.groupby(["course_type", "subcategories"]).apply(lambda x: list(x.index)).to_dict()


In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# calculate pi
X = score_preference.copy()
features = list(X.columns)
X = X.values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=5) 
X_pca = pca.fit_transform(X_scaled)

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_pca) 
score_preference['cluster'] = clusters

def p_score(score_preference, p_number):
    score_mean = pd.DataFrame()

    total_samples = len(score_preference)
    p_weight = []
    for cluster_label in range(p_number):
        cluster_size = len(score_preference[score_preference['cluster'] == cluster_label])
        p_weight.append(cluster_size / total_samples)

    for i, p_weight in enumerate(p_weight):
        print(f"p {i+1}: {p_weight:.2f}")

    for cluster_label in range(p_number):
        cluster_data = score_preference[score_preference['cluster'] == cluster_label] 
        cluster_mean = cluster_data.drop('cluster', axis=1).mean() 
        score_mean[cluster_label] = cluster_mean 

    score_mean = score_mean.T
    print("\nAverage values for each p:")
    print(score_mean)

    return p_weight, score_mean

p_score(score_preference, p_number=3)
p_weight, score_mean = p_score(score_preference, p_number=3)
patron_cluster = dict(enumerate(clusters))

item_names = [meal_data.loc[i, "item_name"] for i in range(num_item)]

pi = { p: { i: 0 for i in range(num_item) } for p in range(3) }
for p in range(3):
    for i in range(num_item):
        dish_name = item_names[i]
        try:
            pi[p][i] = score_mean.loc[p, dish_name]
        except KeyError:
            print(f"在 score_mean 中未找到聚类 {p} 对应的菜品 {dish_name} 数据，默认赋值为0")
            pi[p][i] = 0

# check pi
for p in range(3):
    print(f"聚类 {p} 对各菜品的偏好得分：", pi[p])


c:\Users\18320\AppData\Local\anaconda3\envs\Xpress\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\18320\AppData\Local\anaconda3\envs\Xpress\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


p 1: 0.25
p 2: 0.43
p 3: 0.32

Average values for each p:
     Dish_1    Dish_2    Dish_3    Dish_4    Dish_5    Dish_6    Dish_7  \
0  2.000000  1.285714  1.428571  1.285714  2.000000  1.857143  1.428571   
1  1.833333  1.916667  1.833333  2.083333  2.083333  1.916667  2.083333   
2  2.333333  2.111111  2.333333  2.666667  2.222222  2.333333  2.444444   

     Dish_8    Dish_9   Dish_10  ...  Dish_201  Dish_202  Dish_203  Dish_204  \
0  1.571429  2.142857  2.142857  ...  1.428571  1.571429  1.857143  1.428571   
1  1.583333  1.666667  2.083333  ...  2.333333  1.833333  2.166667  2.000000   
2  2.666667  2.333333  2.555556  ...  2.555556  2.000000  2.333333  2.555556   

   Dish_205  Dish_206  Dish_207  Dish_208  Dish_209  Dish_210  
0  1.714286  1.714286  2.000000  2.285714  2.000000  1.714286  
1  1.833333  2.000000  1.750000  1.833333  2.083333  2.166667  
2  2.555556  2.555556  2.444444  2.555556  2.333333  2.000000  

[3 rows x 210 columns]
p 1: 0.25
p 2: 0.43
p 3: 0.32

Average v

c:\Users\18320\AppData\Local\anaconda3\envs\Xpress\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


model-1

In [9]:
xp.init('C:/xpressmp/bin/xpauth.xpr')

In [10]:
model = xp.problem()
max_time=1000
model.addVariable(x)
model.addVariable(y)
model.addVariable(Q)

dw = {0: 0.25, 1: 0.43, 2: 0.32}
Total_preference_score = xp.Sum(dw[p] * y[i, p, j, t] * pi[p][i] for i in I for p in P for j in M for t in T)
Total_cost = xp.Sum(cost[i] * x[i,j,t] for i in I for j in M for t in T)

model.setObjective(Total_preference_score, sense=xp.maximize)

# A.6 item inclusion
for j in M:
    for l in L:
        for t in T:
            model.addConstraint(
                xp.Sum(x[i, j, t] for i in I_l[l]) == ns[j][l]
            )

# A.7 maximum number of items per subcategory
for t in T:
    for j in M:
        for l in L:
            if l in {"Main dish", "Side", "Soup", "Breakfast"}:    
                for s in S_l:
                    if ns.get((j,l),0) >= 1:
                       model.addConstraint(
                           xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Maximum_Number_of_Items_per_Subcategory_{t}_{j}_{l}_{s}'
                       )


# A.9 course subcategory dependencies
s_l_com = {1:['beef','pork'],2:['fish','shellfish'],3:['grain','pasta']}
for t in T:
    for j in M:
        for l in L:
            if ns.get((j,l),0) >= 2:
                for s in s_l_com[1]:
                    model.addConstraint(xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Course_Sbucategory_Dependencies_{t}_{j}_{l}_{s}')
                for s in s_l_com[2]:
                    model.addConstraint(xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Course_Sbucategory_Dependencies_{t}_{j}_{l}_{s}')
                for s in s_l_com[3]:
                    model.addConstraint(xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Course_Sbucategory_Dependencies_{t}_{j}_{l}_{s}')

# A.10 item dependencies
s_l_dependencies = {1:['beef','poultry','pork'],2:['starch'],3:['vegetable'],4:['grain']}
for t in T:
    for j in M:
        if "Main dish" in L and "Side" in L:
            model.addConstraint(xp.Sum(x[(i, j, t)] for s in s_l_dependencies[1] if (l, s) in I_ls for i in I_ls[(l, s)]) \
                <= xp.Sum(x[(i, j, t)] for s in s_l_dependencies[2] if (l, s) in I_ls for i in I_ls[(l, s)])
            )
        if "Main dish" in L and "Side" in L:
            model.addConstraint(xp.Sum(x[(i, j, t)] for s in s_l_dependencies[3] if (l, s) in I_ls for i in I_ls[(l, s)]) \
                <= xp.Sum(x[(i, j, t)] for s in s_l_dependencies[4] if (l, s) in I_ls for i in I_ls[(l, s)])
            )

# A.11-12 maximum number of servings of items in a menu cycle
nsT_max = 3
for i in I:
    if "Main dish" in L:
        model.addConstraint(
            xp.Sum(x[(i, j, t)] for j in M for t in T) <= nsT_max
        )          

# A.13 maximum number of servings of items in OWPs
def nw(t, T, c):
    return (t + c) % len(T)
W = 3
for l in L:
    for t in T:
        for i in I_l[l]:
            model.addConstraint(
                #xp.Sum(x[(i, j, c)] for j in M for c in range(W)) <= nsOWP[l]
                xp.Sum(x[(i, j, nw(t, T, c))] for j in M for c in range(W)) <= nsOWP[l]    
            )

# A.14-15 maximum/minimum number of course subcategories in OWPs
for t in T:
    for l in L:
        if l == "Main dish":
            for s in S_l[l]:
                model.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 1
                )
                model.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) <= 5
                )
        if l == "Soup":
            for s in S_l[l]:
                model.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 1
                )
        if l == "Dessert":
            if s == "cake":
                model.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 1
                )
            else:
                model.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 0
                )

# A.16 limiting consecutive repetitions of items
G_i = {i: 1 for i in I}
for i in I:
    for t in T:
        if t > G_i[i]:
            model.addConstraint(
                xp.Sum(x[(i, j, nw(t + c - G_i[i], T, 1))] for j in M for c in range(1, G_i[i] + 1)) \
                <= 1 - xp.Sum(x[(i, j, t - G_i[i])] for j in M)
            )

# A.17: item repetition constraints within a day
for l in L:
    for i in I:
        for t in T:
            model.addConstraint(
                xp.Sum(x[(i, j, t)] for j in M) <= nsd[l]
            )

# A.18: diet types: p can choose only the items that offered
for p in P: 
    for j in M:
        for i in I:
            for t in T:
                model.addConstraint(
                    y[(i, p, j, t)]<= x[(i, j, t)]
                )

# A.19: diet types: p is allowed to choose np number of items
for p in P:
    for j in M:
        for l in L:
            for t in T:
                #if ns.get((j,l),0) >= 1:
                    model.addConstraint(
                        xp.Sum(y[(i, p, j, t)] for i in I_l[l]) == np_expanded[p][j][l]
                    )
                    
# A.20-21
for p in P:
    for k in K:
        for t in T:
            nutrient_intake = xp.Sum(
                w[k][i] * y[(i, p, j, t)] 
                for j in M 
                for i in I 
                if (i, p, j, t) in y
            )
            model.addConstraint(
                nutrient_intake <= alpha_max[p][k] + gamma_max[p][k] * Q[(p, k, t)]
            )
for p in P:
    for k in K:
        for t in T:
            nutrient_intake = xp.Sum(
                w[k][i] * y[(i, p, j, t)] 
                for j in M 
                for i in I 
                if (i, p, j, t) in y
            )
            model.addConstraint(
                nutrient_intake >= alpha_min[p][k] - gamma_min[p][k] * Q[(p, k, t)]
            )

# A.23
for p in P:
    for k in K:
        for t in T:
            model.addConstraint(Q[(p, k, t)] <= 0.5)

# A.24 meal cost
breakfast_items = [i for i in I if meal_data.loc[i, 'course_type'] == 'Breakfast']
model.addConstraint(
    xp.Sum(cost[i] * x[i, 'breakfast', t] for i in breakfast_items for t in T) <= 0.2 * Total_cost
)
non_breakfast_courses = [l for l in L if l != 'Breakfast']
lunch_items = [i for l in non_breakfast_courses for i in I_l[l]]
model.addConstraint(
    xp.Sum(cost[i] * x[i, 'lunch', t] for i in lunch_items for t in T) <= 0.4 * Total_cost
)
dinner_items  = [i for l in non_breakfast_courses for i in I_l[l]]
model.addConstraint(
    xp.Sum(cost[i] * x[i, 'dinner', t] for i in dinner_items for t in T) >= 0.4 * Total_cost
)
model.addConstraint(
    xp.Sum(cost[i] * x[i, 'dinner', t] for i in dinner_items for t in T) <= 0.7 * Total_cost
)

print(f"optimal objective:{model.getObjVal()}")



optimal objective:1e+40


In [11]:
model.setControl('maxtime', max_time )
print(model.getControl('maxtime'))
best_bound = model.getAttrib('bestbound')
mip_gap = 150*((model.getObjVal()-best_bound)/model.getObjVal())

xp.setOutputEnabled(True)
model.solve()

1000
FICO Xpress v9.4.2, Hyper, solve started 19:55:46, Mar 13, 2025
Heap usage: 18MB (peak 18MB, 3032KB system)
Maximizing MILP noname using up to 8 threads and up to 7965MB memory, with these control settings:
MAXTIME = 1000
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
     27304 rows        17724 cols       222411 elements     17640 entities
Presolved problem has:
      9363 rows         9863 cols        78260 elements      9863 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 21MB (peak 38MB, 3032KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.04e-01,  1.00e+30] / [ 2.34e-03,  1.95e+00]
  RHS and bounds [min,max] : [ 5.00e-01,  2.30e+03] / [ 2.10e-01,  1.71e+01]
  Objective      [min,max] : [      0.0,       0.0] / [      0.0,       0.0]
Autoscaling applied standard scaling

Symmetric problem: generators: 15, support set: 9480
 Num

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)

In [12]:
xp.setOutputEnabled(True)
model.solve()

FICO Xpress v9.4.2, Hyper, solve started 19:57:03, Mar 13, 2025
Heap usage: 25MB (peak 65MB, 3038KB system)
Maximizing MILP noname using up to 8 threads and up to 7965MB memory, with these control settings:
MAXTIME = 1000
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
     27304 rows        17724 cols       222411 elements     17640 entities
Presolved problem has:
      9398 rows         9863 cols        78470 elements      9863 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 31MB (peak 65MB, 3038KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.04e-01,  1.00e+30] / [ 2.34e-03,  1.95e+00]
  RHS and bounds [min,max] : [ 5.00e-01,  2.30e+03] / [ 2.10e-01,  1.71e+01]
  Objective      [min,max] : [      0.0,       0.0] / [      0.0,       0.0]
Autoscaling applied standard scaling

Symmetric problem: generators: 15, support set: 9480
 Number o

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)

model 2

stage 1

In [44]:
# add variables and parameters
max_time=1000
WN = 3
N = 3 # Maximum daily edible parameters
M_ld = ['lunch','dinner']
nm = {0: 14, 1: 10, 2: 12}  # Total number of swipes per customer type---1 week
wp = {                          # wp[p][k]
    0: {'calories': 0.5,'cholesterol':0.2,'sodium':0.1, 'dietary_fiber': 0.2},  # man - high calories
    1: {'calories': 0.2, 'cholesterol':0.2,'sodium':0.2, 'dietary_fiber': 0.4},  #Female - high dietary_fiber
    2: {'calories': 0.25, 'cholesterol':0.25,'sodium':0.2, 'dietary_fiber': 0.25}  # Blend type, balance nutrition and taste
}
u = {
    (p, j, t): xp.var(
        vartype=xp.binary,
        name=f'u_{p}_{j}_{t}'
    )
    for p in P for j in M_ld for t in T
}
Qmax = {
    p: xp.var(
        vartype=xp.continuous,
        lb=0,
        name=f'Qmax_{p}'
    )
    for p in P 
}

stage1 = xp.problem()
stage1.addVariable(x)
stage1.addVariable(y)
stage1.addVariable(Qmax)
stage1.addVariable(Q)
stage1.addVariable(u)

Total_variation_1 = xp.Sum(Qmax[p]/1 for p in P)
Total_cost = xp.Sum(cost[i] * x[i,j,t] for i in I for j in M for t in T)

stage1.setObjective(Total_variation_1, sense=xp.minimize)

# A.6 item inclusion
for j in M:
    for l in L:
        for t in T:
            stage1.addConstraint(
                xp.Sum(x[i, j, t] for i in I_l[l]) == ns[j][l]
            )

# A.7 maximum number of items per subcategory
for t in T:
    for j in M:
        for l in L:
            if l in {"Main dish", "Side", "Soup", "Breakfast"}:    
                for s in S_l:
                    if ns.get((j,l),0) >= 1:
                       stage1.addConstraint(
                           xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Maximum_Number_of_Items_per_Subcategory_{t}_{j}_{l}_{s}'
                       )


# A.9 course subcategory dependencies
s_l_com = {1:['beef','pork'],2:['fish','shellfish'],3:['grain','pasta']}
for t in T:
    for j in M:
        for l in L:
            if ns.get((j,l),0) >= 2:
                for s in s_l_com[1]:
                    stage1.addConstraint(xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Course_Sbucategory_Dependencies_{t}_{j}_{l}_{s}')
                for s in s_l_com[2]:
                    stage1.addConstraint(xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Course_Sbucategory_Dependencies_{t}_{j}_{l}_{s}')
                for s in s_l_com[3]:
                    stage1.addConstraint(xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Course_Sbucategory_Dependencies_{t}_{j}_{l}_{s}')

# A.10 item dependencies
s_l_dependencies = {1:['beef','poultry','pork'],2:['starch'],3:['vegetable'],4:['grain']}
for t in T:
    for j in M:
        if "Main dish" in L and "Side" in L:
            stage1.addConstraint(xp.Sum(x[(i, j, t)] for s in s_l_dependencies[1] if (l, s) in I_ls for i in I_ls[(l, s)]) \
                <= xp.Sum(x[(i, j, t)] for s in s_l_dependencies[2] if (l, s) in I_ls for i in I_ls[(l, s)])
            )
        if "Main dish" in L and "Side" in L:
            stage1.addConstraint(xp.Sum(x[(i, j, t)] for s in s_l_dependencies[3] if (l, s) in I_ls for i in I_ls[(l, s)]) \
                <= xp.Sum(x[(i, j, t)] for s in s_l_dependencies[4] if (l, s) in I_ls for i in I_ls[(l, s)])
            )

# A.11-12 maximum number of servings of items in a menu cycle
nsT_max = 3
for i in I:
    if "Main dish" in L:
        stage1.addConstraint(
            xp.Sum(x[(i, j, t)] for j in M for t in T) <= nsT_max
        )          

# A.13 maximum number of servings of items in OWPs
def nw(t, T, c):
    return (t + c) % len(T)
W = 3
for l in L:
    for t in T:
        for i in I_l[l]:
            stage1.addConstraint(
                #xp.Sum(x[(i, j, c)] for j in M for c in range(W)) <= nsOWP[l]
                xp.Sum(x[(i, j, nw(t, T, c))] for j in M for c in range(W)) <= nsOWP[l]    
            )

# A.14-15 maximum/minimum number of course subcategories in OWPs
for t in T:
    for l in L:
        if l == "Main dish":
            for s in S_l[l]:
                stage1.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 1
                )
                stage1.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) <= 5
                )
        if l == "Soup":
            for s in S_l[l]:
                stage1.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 1
                )
        if l == "Dessert":
            if s == "cake":
                stage1.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 1
                )
            else:
                stage1.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 0
                )

# A.16 limiting consecutive repetitions of items
G_i = {i: 1 for i in I}
for i in I:
    for t in T:
        if t > G_i[i]:
            stage1.addConstraint(
                xp.Sum(x[(i, j, nw(t + c - G_i[i], T, 1))] for j in M for c in range(1, G_i[i] + 1)) \
                <= 1 - xp.Sum(x[(i, j, t - G_i[i])] for j in M)
            )

# A.17: item repetition constraints within a day
for l in L:
    for i in I:
        for t in T:
            stage1.addConstraint(
                xp.Sum(x[(i, j, t)] for j in M) <= nsd[l]
            )

# A.18: diet types: p can choose only the items that offered
for p in P: 
    for j in M:
        for i in I:
            for t in T:
                stage1.addConstraint(
                    y[(i, p, j, t)]<= x[(i, j, t)]
                )

# A.19: diet types: p is allowed to choose np number of items
for p in P:
    for j in M:
        for l in L:
            for t in T:
                #if ns.get((j,l),0) >= 1:
                    stage1.addConstraint(
                        xp.Sum(y[(i, p, j, t)] for i in I_l[l]) == np_expanded[p][j][l]
                    )
                    
# A.20-21
for p in P:
    for k in K:
        for t in T:
            nutrient_intake = xp.Sum(
                w[k][i] * y[(i, p, j, t)] 
                for j in M 
                for i in I 
                if (i, p, j, t) in y
            )
            stage1.addConstraint(
                nutrient_intake <= alpha_max[p][k] + gamma_max[p][k] * Q[(p, k, t)]
            )
for p in P:
    for k in K:
        for t in T:
            nutrient_intake = xp.Sum(
                w[k][i] * y[(i, p, j, t)] 
                for j in M 
                for i in I 
                if (i, p, j, t) in y
            )
            stage1.addConstraint(
                nutrient_intake >= alpha_min[p][k] - gamma_min[p][k] * Q[(p, k, t)]
            )

# A.23
for p in P:
    for k in K:
        for t in T:
            stage1.addConstraint(Q[(p, k, t)] <= 0.5)

# A.24 meal cost
breakfast_items = [i for i in I if meal_data.loc[i, 'course_type'] == 'Breakfast']
stage1.addConstraint(
    xp.Sum(cost[i] * x[i, 'breakfast', t] for i in breakfast_items for t in T) <= 0.2 * Total_cost
)
non_breakfast_courses = [l for l in L if l != 'Breakfast']
lunch_items = [i for l in non_breakfast_courses for i in I_l[l]]
stage1.addConstraint(
    xp.Sum(cost[i] * x[i, 'lunch', t] for i in lunch_items for t in T) <= 0.4 * Total_cost
)
dinner_items  = [i for l in non_breakfast_courses for i in I_l[l]]
stage1.addConstraint(
    xp.Sum(cost[i] * x[i, 'dinner', t] for i in dinner_items for t in T) >= 0.4 * Total_cost
)
stage1.addConstraint(
    xp.Sum(cost[i] * x[i, 'dinner', t] for i in dinner_items for t in T) <= 0.7 * Total_cost
)

# C.1
for p in P:
    for j in M_ld:
            for t in T:
                stage1.addConstraint(
                        xp.Sum(y[(i, p, j, t)] for i in I) <= 3 * u[(p, j, t)]
                    )

# C.2
for p in P:
    stage1.addConstraint(xp.Sum(u[(p,j,t)] for j in M_ld for t in T) == nm[p])

# C.3
def nw(t, T, c):
    return (t + c) % len(T)
for p in P:
    for t in T:
            stage1.addConstraint(
                    xp.Sum(wp[p][k]* Q[(p,k, nw(t, T, c))] for k in K for c in range(1, WN+1)) <= Qmax[p]
                )

KeyError: (169, 'breakfast', 0)

In [34]:
stage1.setControl('maxtime', max_time )
print(stage1.getControl('maxtime'))
best_bound = stage1.getAttrib('bestbound')
mip_gap = 150*((stage1.getObjVal()-best_bound)/stage1.getObjVal())

xp.setOutputEnabled(True)
stage1.solve()

1000
FICO Xpress v9.4.2, Hyper, solve started 22:52:31, Mar 12, 2025
Heap usage: 11MB (peak 11MB, 2820KB system)
Minimizing MILP noname using up to 8 threads and up to 7965MB memory, with these control settings:
MAXTIME = 1000
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
     17363 rows        11049 cols       141996 elements     10962 entities
 
 
The problem is infeasible due to row R17341
Presolve finished in 0 seconds
Heap usage: 9659KB (peak 21MB, 5127KB system)
 *** Search completed ***
Problem is integer infeasible
  Solution time / primaldual integral :      0.11s/ 100.000000%
  Number of solutions found / nodes   :         0 /         0


(<SolveStatus.COMPLETED: 3>, <SolStatus.INFEASIBLE: 3>)

In [36]:
stage1.write("model.lp")

stage 2

In [ ]:
stage2 = xp.problem()
stage2.addVariable(x)
stage2.addVariable(y)

Total_variation_2 = xp.Sum((300-y[i, p, j, t] * pi[p][i])/300  for i in I for p in P for j in M for t in T)

stage2.setObjective(Total_variation_2, sense=xp.minimize)

# A.6 item inclusion
for j in M:
    for l in L:
        for t in T:
            stage2.addConstraint(
                xp.Sum(x[i, j, t] for i in I_l[l]) == ns[j][l]
            )

# A.7 maximum number of items per subcategory
for t in T:
    for j in M:
        for l in L:
            if l in {"Main dish", "Side", "Soup", "Breakfast"}:    
                for s in S_l:
                    if ns.get((j,l),0) >= 1:
                       stage2.addConstraint(
                           xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Maximum_Number_of_Items_per_Subcategory_{t}_{j}_{l}_{s}'
                       )


# A.9 course subcategory dependencies
s_l_com = {1:['beef','pork'],2:['fish','shellfish'],3:['grain','pasta']}
for t in T:
    for j in M:
        for l in L:
            if ns.get((j,l),0) >= 2:
                for s in s_l_com[1]:
                    stage2.addConstraint(xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Course_Sbucategory_Dependencies_{t}_{j}_{l}_{s}')
                for s in s_l_com[2]:
                    stage2.addConstraint(xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Course_Sbucategory_Dependencies_{t}_{j}_{l}_{s}')
                for s in s_l_com[3]:
                    stage2.addConstraint(xp.Sum(x[i,j,t] for i in I_ls[l][s]) <=1, name=f'Course_Sbucategory_Dependencies_{t}_{j}_{l}_{s}')

# A.10 item dependencies
s_l_dependencies = {1:['beef','poultry','pork'],2:['starch'],3:['vegetable'],4:['grain']}
for t in T:
    for j in M:
        if "Main dish" in L and "Side" in L:
            stage2.addConstraint(xp.Sum(x[(i, j, t)] for s in s_l_dependencies[1] if (l, s) in I_ls for i in I_ls[(l, s)]) \
                <= xp.Sum(x[(i, j, t)] for s in s_l_dependencies[2] if (l, s) in I_ls for i in I_ls[(l, s)])
            )
        if "Main dish" in L and "Side" in L:
            stage2.addConstraint(xp.Sum(x[(i, j, t)] for s in s_l_dependencies[3] if (l, s) in I_ls for i in I_ls[(l, s)]) \
                <= xp.Sum(x[(i, j, t)] for s in s_l_dependencies[4] if (l, s) in I_ls for i in I_ls[(l, s)])
            )

# A.11-12 maximum number of servings of items in a menu cycle
nsT_max = 3
for i in I:
    if "Main dish" in L:
        stage1.addConstraint(
            xp.Sum(x[(i, j, t)] for j in M for t in T) <= nsT_max
        )          

# A.13 maximum number of servings of items in OWPs
def nw(t, T, c):
    return (t + c) % len(T)
W = 3
for l in L:
    for t in T:
        for i in I_l[l]:
            stage2.addConstraint(
                #xp.Sum(x[(i, j, c)] for j in M for c in range(W)) <= nsOWP[l]
                xp.Sum(x[(i, j, nw(t, T, c))] for j in M for c in range(W)) <= nsOWP[l]    
            )

# A.14-15 maximum/minimum number of course subcategories in OWPs
for t in T:
    for l in L:
        if l == "Main dish":
            for s in S_l[l]:
                stage2.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 1
                )
                stage2.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) <= 5
                )
        if l == "Soup":
            for s in S_l[l]:
                stage2.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 1
                )
        if l == "Dessert":
            if s == "cake":
                stage2.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 1
                )
            else:
                stage2.addConstraint(
                    xp.Sum(x[(i, j, nw(t, T, c))] for i in I_ls.get((l, s), []) for j in M for c in range(W)) >= 0
                )

# A.16 limiting consecutive repetitions of items
G_i = {i: 1 for i in I}
for i in I:
    for t in T:
        if t > G_i[i]:
            stage2.addConstraint(
                xp.Sum(x[(i, j, nw(t + c - G_i[i], T, 1))] for j in M for c in range(1, G_i[i] + 1)) \
                <= 1 - xp.Sum(x[(i, j, t - G_i[i])] for j in M)
            )

# A.17: item repetition constraints within a day
for l in L:
    for i in I:
        for t in T:
            stage2.addConstraint(
                xp.Sum(x[(i, j, t)] for j in M) <= nsd[l]
            )

# A.18: diet types: p can choose only the items that offered
for p in P: 
    for j in M:
        for i in I:
            for t in T:
                stage2.addConstraint(
                    y[(i, p, j, t)]<= x[(i, j, t)]
                )

# A.19: diet types: p is allowed to choose np number of items
for p in P:
    for j in M:
        for l in L:
            for t in T:
                #if ns.get((j,l),0) >= 1:
                    stage2.addConstraint(
                        xp.Sum(y[(i, p, j, t)] for i in I_l[l]) == np_expanded[p][j][l]
                    )
                    
# A.20-21
for p in P:
    for k in K:
        for t in T:
            nutrient_intake = xp.Sum(
                w[k][i] * y[(i, p, j, t)] 
                for j in M 
                for i in I 
                if (i, p, j, t) in y
            )
            stage2.addConstraint(
                nutrient_intake <= alpha_max[p][k] + gamma_max[p][k] * Q[(p, k, t)]
            )
for p in P:
    for k in K:
        for t in T:
            nutrient_intake = xp.Sum(
                w[k][i] * y[(i, p, j, t)] 
                for j in M 
                for i in I 
                if (i, p, j, t) in y
            )
            stage2.addConstraint(
                nutrient_intake >= alpha_min[p][k] - gamma_min[p][k] * Q[(p, k, t)]
            )

# A.23
for p in P:
    for k in K:
        for t in T:
            stage2.addConstraint(Q[(p, k, t)] <= 0.3)

# A.24 meal cost
breakfast_items = [i for i in I if meal_data.loc[i, 'course_type'] == 'Breakfast']
stage2.addConstraint(
    xp.Sum(cost[i] * x[i, 'breakfast', t] for i in breakfast_items for t in T) <= 0.2 * Total_cost
)
non_breakfast_courses = [l for l in L if l != 'Breakfast']
lunch_items = [i for l in non_breakfast_courses for i in I_l[l]]
stage2.addConstraint(
    xp.Sum(cost[i] * x[i, 'lunch', t] for i in lunch_items for t in T) <= 0.4 * Total_cost
)
dinner_items  = [i for l in non_breakfast_courses for i in I_l[l]]
stage2.addConstraint(
    xp.Sum(cost[i] * x[i, 'dinner', t] for i in dinner_items for t in T) >= 0.4 * Total_cost
)
stage2.addConstraint(
    xp.Sum(cost[i] * x[i, 'dinner', t] for i in dinner_items for t in T) <= 0.7 * Total_cost
)

# C.1


# C.2


# C.3


# C.5
f_Q = 
model.addConstraint(xp.Sum(Qmax[p]/2 for i in I for p in P) <= f_Q)

In [ ]:
stage2.setControl('maxtime', max_time )
print(stage2.getControl('maxtime'))
best_bound = stage2.getAttrib('bestbound')
mip_gap = 150*((stage2.getObjVal()-best_bound)/stage2.getObjVal())

xp.setOutputEnabled(True)
stage2.solve()

1000
FICO Xpress v9.4.2, Hyper, solve started 18:53:32, Mar 11, 2025
Heap usage: 4141KB (peak 4141KB, 9050KB system)
Minimizing MILP noname using up to 8 threads and up to 7965MB memory, with these control settings:
MAXTIME = 1000
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
         0 rows        10923 cols            0 elements     10920 entities
Presolved problem has:
         0 rows        10923 cols            0 elements     10920 entities
Presolve finished in 0 seconds
Heap usage: 5763KB (peak 6020KB, 9050KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [      0.0,       0.0] / [      0.0,       0.0]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 1.36e+03,  1.36e+03] / [ 1.36e+03,  1.36e+03]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 5.4GB
 *** Solut

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)

In [18]:
y_solution = {key: model.getSolution(var) for key, var in y.items()}

# show the items that selected
selected_y = {key: value for key, value in y_solution.items() if value > 0.5}

df_y = pd.DataFrame([
    {"Item": key[0], "Guest": key[1], "Meal": key[2], "Time": key[3], "Value": value}
    for key, value in selected_y.items()
])
df_y.to_csv("3.11y_result.csv", index=False)

preference_scores = {}

for t in T:
    score = 0
    for p in P:
        for j in M:
            for i in I:
                score += y_solution[(i, p, j, t)] * pi[p][i]
    preference_scores[t] = score
    
df = pd.DataFrame(list(preference_scores.items()), columns=['Day', 'Preference Score'])

df.to_csv('3.11daily_preference_scores.csv', index=False)